<a href="https://colab.research.google.com/github/harry-graves/Aria_ORI/blob/main/trajectory_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trajectory Conversion
This script converts both the OpenVINS trajectory format and the Aria MPS trajectory formats to the more convenient TUM format. Note that the MPS closed loop and open loop trajectories have differing formats, so the correct type must be specified when calling `convert_mps_to_tum`.

In [ ]:
import os

# To convert OpenVINS trajectories (.txt) to .tum format

def convert_ov_to_tum(input_file):
    # Change the extension of the input file from .txt to .tum
    output_file = os.path.splitext(input_file)[0] + '.tum'

    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        first_line = True  # Flag to skip the header
        for line in infile:
            if first_line:
                first_line = False  # Skip the first line (header)
                continue

            if line.startswith("#") or not line.strip():
                continue  # Skip comments and empty lines

            data = line.strip().split()
            timestamp = data[0]
            # Extract quaternion and position components
            q_x, q_y, q_z, q_w = data[1:5]  # Quaternion
            p_x, p_y, p_z = map(float, data[5:8])  # Position

            # Write in TUM format: timestamp, p_x, p_y, p_z, q_x, q_y, q_z, q_w
            outfile.write(f"{timestamp} {p_x} {p_y} {p_z} {q_x} {q_y} {q_z} {q_w}\n")


# To convert Aria MPS trajectories (.csv) to .tum format

def convert_mps_to_tum(input_file,loop="closed"):
    # Change the extension of the input file from .txt or .csv to .tum
    output_file = os.path.splitext(input_file)[0] + '.tum'

    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        first_line = True  # Flag to skip the header
        for line in infile:
            if first_line:
                first_line = False  # Skip the first line (header)
                continue

            if not line.strip():
                continue  # Skip empty lines

            data = line.strip().split(',')

            # The closed loop and open loop outputs have slightly differnet formats
            # For open loop, the timestamp is in column 0, vs column 1 for closed loop
            if loop == "open":
                timestamp_us = float(data[0])  # tracking_timestamp_us
            else:
                timestamp_us = float(data[1])  # tracking_timestamp_us
            # Convert microseconds to seconds for TUM format
            timestamp_s = timestamp_us / 1e6

            # Extract position components
            p_x, p_y, p_z = map(float, data[3:6])  # Convert to float

            # Extract quaternion components
            q_x, q_y, q_z, q_w = data[6:10]

            # Write in TUM format: timestamp, p_x, p_y, p_z, q_x, q_y, q_z, q_w
            outfile.write(f"{timestamp_s:.6f} {p_x} {p_y} {p_z} {q_x} {q_y} {q_z} {q_w}\n")


# Example usage
input_filepath = 'ov_estimate.txt'
convert_ov_to_tum(input_filepath)

input_filepath = 'open_loop_trajectory.csv'
convert_mps_to_tum(input_filepath,loop="open")

input_filepath = 'closed_loop_trajectory.csv'
convert_mps_to_tum(input_filepath,loop="closed")